# Usage example

### Setting input dataset and output directory

Required: dataset_name, model_name, explanation_name

Possible values:

In [1]:
from src import strings_to_classes_mappings

print("Possible datasets:", list(strings_to_classes_mappings.datasets_mapping.keys()))
print("Possible models:", list(strings_to_classes_mappings.models_mapping.keys()))
print("Possible explanations:", list(strings_to_classes_mappings.explanations_mapping.keys()))

Possible datasets: ['imagenette2']
Possible models: ['DeiT_S', 'DeiT_T', 'DenseNet121', 'EfficientNet_B3', 'EfficientNet_B4', 'ConvNeXtV2_Nano', 'PyramidViT_V2_B2', 'MobileNetV3', 'Swin_T', 'ResNet18', 'ResNet50', 'ResNeXt50', 'Res2Net50', 'ViT_B_32']
Possible explanations: ['GradCAM', 'KernelSHAP']


**imagenette2** download from [here](https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz) (or check for link [here](https://github.com/fastai/imagenette)) and extract to `../datasets/imagenette2`.

Input and output folders:
- root_images - passed to torchvision ImageFolder, the folder should contain subfolders with images for different classes
- root_explanations - path to the folder where folders with explanations will be created

### Example usage:

First, compute explanations for at least 2 models.

In [2]:
from src.compute_explanations import compute_explanations

compute_explanations(dataset_name='imagenette2', model_name='ResNet18', explanation_name='GradCAM', 
                     number_of_batches_to_process=2, batch_size=4, shuffle=True, 
                     device="cuda",                                     # <- used by pytorch, as in model.to(device)
                     root_images="../datasets/imagenette2/train",       # <- folder from which torchvision's ImageFolder will load images
                     root_explanations="../explanations")               # <- here, the explanations will be saved


compute_explanations(dataset_name='imagenette2', model_name='ResNet50', explanation_name='GradCAM',  # <- another model
                     number_of_batches_to_process=2, batch_size=4, shuffle=True, 
                     device="cuda",                                     
                     root_images="../datasets/imagenette2/train",       
                     root_explanations="../explanations")               

Loaded model: https://huggingface.co/timm/resnet18d.ra2_in1k


Computing explanations for batches: 100%|██████████| 2/2 [00:01<00:00,  1.95it/s]


Loaded model: https://huggingface.co/timm/resnet50d.ra2_in1k


Computing explanations for batches: 100%|██████████| 2/2 [00:01<00:00,  1.59it/s]


Then, load explanations and compute similarity metrics.

In [3]:
from src.compare_explanations import compare_explanations, cosine_similarity, radial_basis_function
from src.load_explanations import load_explanations_of_many_models

models_list = ['ResNet18','ResNet50']
explanations_many_models = load_explanations_of_many_models('imagenette2', models_list, 'GradCAM', root_explanations="../explanations/rubbish")

similarity_df_cosine = compare_explanations(explanations_many_models, comparison_function=cosine_similarity)
print("\nCosine similarity:")
print(similarity_df_cosine, '\n')

similarity_df_rbf = compare_explanations(explanations_many_models, comparison_function=radial_basis_function)
print("Radial basis function similarity:")
print(similarity_df_rbf)

Loading explanations: 100%|██████████| 8/8 [00:00<00:00, 433.40it/s]

Loaded 8 common explanations for each model.

Cosine similarity:
          ResNet18  ResNet50
ResNet18  1.000001  0.858149
ResNet50  0.858149  1.000001 

Radial basis function similarity:
          ResNet18  ResNet50
ResNet18       1.0  0.072918
ResNet50  0.072918       1.0


You can limit the compared explanations to only those, for which model's predictions were the same.

In [4]:
from src.compare_explanations import count_same_predictions

count_of_same_predictions = count_same_predictions(explanations_many_models)

print("Count of same predictions:")
print(count_of_same_predictions, '\n')

similarity_df_cosine = compare_explanations(explanations_many_models, comparison_function=cosine_similarity, compare_only_explanations_with_same_predictions=True)
similarity_df_rbf = compare_explanations(explanations_many_models, comparison_function=radial_basis_function, compare_only_explanations_with_same_predictions=True)

print("Cosine similarity:")
print(similarity_df_cosine, '\n')

print("Radial basis function similarity:")
print(similarity_df_rbf)

Count of same predictions:
         ResNet18 ResNet50
ResNet18        8        6
ResNet50        6        8 

Cosine similarity:
          ResNet18  ResNet50
ResNet18  1.000001  0.915739
ResNet50  0.915739  1.000001 

Radial basis function similarity:
         ResNet18 ResNet50
ResNet18      1.0  0.09722
ResNet50  0.09722      1.0


#### CPU vs CUDA

In [5]:
from src.compute_explanations import compute_explanations

print("With CPU:")
compute_explanations('imagenette2', 'ResNet18', 'GradCAM', number_of_batches_to_process=8, batch_size=8, shuffle=True, device="cpu")

print("With GPU:")
compute_explanations('imagenette2', 'ResNet18', 'GradCAM', number_of_batches_to_process=8, batch_size=8, shuffle=True, device="cuda")

With CPU:
Loaded model: https://huggingface.co/timm/resnet18d.ra2_in1k


Computing explanations for batches: 100%|██████████| 8/8 [00:13<00:00,  1.66s/it]


With GPU:
Loaded model: https://huggingface.co/timm/resnet18d.ra2_in1k


Computing explanations for batches: 100%|██████████| 8/8 [00:20<00:00,  2.58s/it]


## Full computations - example

First, choose models:

In [6]:
# all models
# models_list = list(strings_to_classes_mappings.models_mapping.keys())

# only these models (example)
models_list = ['ResNet18','ResNet50', 'ViT_B_32', 'Swin_T', 'DeiT_S', 'EfficientNet_B3']

Choose also other parameters:

In [7]:
compute_explanations_parameters = {
    
    # Increase the following to analyze more images.
    "number_of_batches_to_process" : 4, 
    "batch_size" : 4,

    "device" : "cuda", # for torch on cuda gpu, or 'cpu' for cpu etc
    "shuffle" : True, # for dataloader
}

#### GradCAM

In [8]:
from src.compute_explanations import compute_explanations

for model in models_list:
    compute_explanations('imagenette2', model, 'GradCAM', **compute_explanations_parameters)

Loaded model: https://huggingface.co/timm/resnet18d.ra2_in1k


Computing explanations for batches: 100%|██████████| 4/4 [00:06<00:00,  1.55s/it]


Loaded model: https://huggingface.co/timm/resnet50d.ra2_in1k


Computing explanations for batches: 100%|██████████| 4/4 [00:15<00:00,  3.78s/it]


Loaded model: https://huggingface.co/timm/vit_base_patch32_224.augreg_in21k_ft_in1k


...

Computing explanations for batches: 100%|██████████| 4/4 [00:22<00:00,  5.51s/it]


Loaded model: https://huggingface.co/timm/deit_small_patch16_224.fb_in1k


Computing explanations for batches: 100%|██████████| 4/4 [00:16<00:00,  4.24s/it]


Loaded model: https://huggingface.co/timm/efficientnet_b3.ra2_in1k


Computing explanations for batches: 100%|██████████| 4/4 [00:17<00:00,  4.37s/it]


In [9]:
from src.compare_explanations import compare_explanations, cosine_similarity
from src.load_explanations import load_explanations_of_many_models

explanations_many_models = load_explanations_of_many_models('imagenette2', models_list, 'GradCAM')

similarity_df = compare_explanations(explanations_many_models, comparison_function=cosine_similarity)
similarity_df.to_csv("similarity_matrix_gradcam.csv")
similarity_df

Loading explanations: 100%|██████████| 16/16 [00:00<00:00, 58.64it/s]


Loaded 16 common explanations for each model.


,ResNet18,ResNet50,ViT_B_32,Swin_T,DeiT_S,EfficientNet_B3
ResNet18,1.000001,0.85677,0.584933,0.802545,0.66538,0.799556
ResNet50,0.85677,1.000001,0.612756,0.862724,0.645426,0.822643
ViT_B_32,0.584933,0.612756,1.0,0.586312,0.546911,0.610318
Swin_T,0.802545,0.862724,0.586312,1.0,0.659092,0.773872
DeiT_S,0.66538,0.645426,0.546911,0.659092,1.0,0.64866
EfficientNet_B3,0.799556,0.822643,0.610318,0.773872,0.64866,1.0


#### KernelSHAP

In [10]:
from src.compute_explanations import compute_explanations

for model in models_list:
    compute_explanations('imagenette2', model, 'KernelSHAP', **compute_explanations_parameters)

Loaded model: https://huggingface.co/timm/resnet18d.ra2_in1k


Computing explanations for batches: 100%|██████████| 4/4 [03:54<00:00, 58.64s/it]


Loaded model: https://huggingface.co/timm/resnet50d.ra2_in1k


Computing explanations for batches: 100%|██████████| 4/4 [10:02<00:00, 150.66s/it]


Loaded model: https://huggingface.co/timm/vit_base_patch32_224.augreg_in21k_ft_in1k


...

Computing explanations for batches: 100%|██████████| 4/4 [02:23<00:00, 35.89s/it]


Loaded model: https://huggingface.co/timm/deit_small_patch16_224.fb_in1k


Computing explanations for batches: 100%|██████████| 4/4 [01:31<00:00, 22.91s/it]


Loaded model: https://huggingface.co/timm/efficientnet_b3.ra2_in1k


Computing explanations for batches: 100%|██████████| 4/4 [02:43<00:00, 40.95s/it]


In [11]:
from src.compare_explanations import compare_explanations, cosine_similarity
from src.load_explanations import load_explanations_of_many_models

explanations_many_models = load_explanations_of_many_models('imagenette2', models_list, 'KernelSHAP')

similarity_df = compare_explanations(explanations_many_models, comparison_function=cosine_similarity)
similarity_df.to_csv("similarity_matrix_kernelshap.csv")
similarity_df

Loading explanations: 100%|██████████| 16/16 [00:00<00:00, 27.76it/s]


Loaded 16 common explanations for each model.


,ResNet18,ResNet50,ViT_B_32,Swin_T,DeiT_S,EfficientNet_B3
ResNet18,0.999998,0.876981,0.870733,0.879172,0.859793,0.883495
ResNet50,0.876981,1.000006,0.882726,0.899087,0.885738,0.888241
ViT_B_32,0.870733,0.882726,0.999998,0.899373,0.899415,0.887347
Swin_T,0.879172,0.899087,0.899373,1.000006,0.911456,0.900766
DeiT_S,0.859793,0.885738,0.899415,0.911456,0.999996,0.898725
EfficientNet_B3,0.883495,0.888241,0.887347,0.900766,0.898725,1.0
